In [1]:
''' 

Submitter : JESOON KANG, 20170937
Date : 2019. 10. 


    Assignment 4. 

-   -


'''
import matplotlib.pyplot as plt
import math
import numpy as np
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import torchvision
import os


In [2]:

#### Section 1. #### This Section is bringed Data_import_ex.py file.

# Image Data import & resize

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = '../data/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=0)  


validation_data_path = '../data/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=100, shuffle=True, num_workers=0)  

#### Section 1 END ####


In [3]:

# sigmoid Function.
def sigmoid(z) :
    return 1 / (1 + torch.pow(math.e,-z)+0.000001)

def tanh(z) :
    ret = (2 / (1 + torch.pow(math.e,-2*z) + 0.000001)) - 1
    return ret

def derv_tanh(z) :
    ret = (1 - (tanh(z)**2))
    return ret

def relu(z) :
    ret = z.clone().detach()
    #print(ret)
    tmp = torch.zeros_like(ret)
    ret = torch.where(ret<=0,tmp,ret)
    return ret
                                                                  
def derv_relu(z) :
    ret = z.clone().detach()
    tmp_1 = torch.zeros_like(ret)
    tmp_2 = torch.ones_like(ret)
    ret = torch.where(ret<0,tmp_1,tmp_2)
    #print(ret)
    return ret
    
            
def leakly_relu(z) :
    ret = z.clone().detach()
    tmp1 = torch.ones_like(ret)
    ret = torch.where(ret<0,ret*0.01,ret)
    return ret

def derv_leakly_relu(z) :
    ret = z.clone().detach()
    tmp1 = torch.ones_like(ret)
    ret = torch.where(ret<0, tmp1*0.01,tmp1)
    
    return ret

def get_activation(z,type) :
    if (type == 0) :
        return sigmoid(z)
    elif (type == 1) :
        return tanh(z)
    elif (type == 2) :
        return relu(z)
    elif (type == 3) :
        return leakly_relu(z)
    
    else :
        print("Error, get_activation")
        return 0

def get_derv_activation(z,type) :
    if (type == 0) :
        return sigmoid(z)*(1-sigmoid(z))
    elif (type == 1) :
        return derv_tanh(z)
    elif (type == 2) :
        return derv_relu(z)
    elif (type == 3) :
        return derv_leakly_relu(z)    
    else :
        print("Error, get_derv_activation")
        return 0                                                                  

In [4]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce GTX 1050
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [5]:
def get_loss(y,a) :
    #print(y,a)
    ret = -(torch.div(y,a+0.000001) - torch.div(1-y,1-a+0.000001))
    
    return ret

BATCH_SIZE = 100
FEATURE_SIZE = 10000
class ML :
    def __init__(self,act1,act2,act3,L1_size,L2_size,L3_size,lr,min_loss_diff) :
        self.act_type_1 = act1
        self.act_type_2 = act2
        self.act_type_3 = act3 
        self.l1_size = L1_size
        self.l2_size = L2_size
        self.l3_size = L3_size
        self.feature_size = 10000
        self.epoch = 0
        self.lr = lr
        self.min_loss_diff = min_loss_diff
        
        self.init_weights()
        
        print("ML Object initialized")
        self.iter = 0

        self.val_acc_log = []
        self.val_loss_log = []
        self.train_acc_log = []
        self.train_loss_log = []
        self.epoch_log = []
        self.val_acc_log.append(0)
        self.val_loss_log.append(0)
        self.train_acc_log.append(0)
        self.train_loss_log.append(0)
        self.epoch_log.append(0)
        
    def init_weights(self) :
        self.w_1 = torch.FloatTensor(self.l1_size,self.feature_size).uniform_(-1,1).to(device)
        
        self.b_1 = torch.FloatTensor(1,self.l1_size).uniform_(-1,1).to(device)
        
        #torch.FloatTensor(a, b).uniform_(r1, r2)
        
        self.w_2 = torch.FloatTensor(self.l2_size,self.l1_size).uniform_(-1,1).to(device)
        self.b_2 = torch.FloatTensor(1,self.l2_size).uniform_(-1,1).to(device)

        self.w_3 = torch.FloatTensor(self.l3_size,self.l2_size).uniform_(-1,1).to(device)
        self.b_3 = torch.FloatTensor(1,self.l3_size).uniform_(-1,1).to(device)

        

        
    def training(self) :
        
        epoch = 0
        for epoch in range(0,1000000):
            train_acc_log_tmp = []
            train_loss_log_tmp = []
            val_acc_log_tmp = []
            val_loss_log_tmp = []
            

        # load training images of the batch size for every iteration
            for i, data in enumerate(trainloader):
                
                # inputs is the images
                # labels is the class of the image
                inputs, labels = data
                inputs, labels = inputs.float() , labels.float()
                # if you don't change the image size, it will be [batch_size, 1, 100, 100]

                # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
                
                self.t_data_batch = torch.squeeze(torch.FloatTensor(data[0]),3)
                
                self.t_data_batch = self.t_data_batch.view(len(data[0]),FEATURE_SIZE).to(device)
                self.z_1 = torch.matmul(self.t_data_batch,self.w_1.T) + self.b_1
                #print(self.z_1)
                self.a_1 = get_activation(self.z_1,self.act_type_1)
                #print(self.a_1)
                
                self.z_2 = torch.matmul(self.a_1,self.w_2.T) + self.b_2

                self.a_2 = get_activation(self.z_2,self.act_type_2) # 1027 x 50

                self.z_3 = torch.matmul(self.a_2,self.w_3.T) + self.b_3
                #print(self.z_3)
                self.a_3 = get_activation(self.z_3,self.act_type_3) # 1027 x 1
            
                #print(self.a_3)
                self.t_yh_batch = data[1].float().unsqueeze(1).to(device)

                #print(self.a_3,self.t_yh_batch)
                acc = self.get_acc(self.a_3,self.t_yh_batch)
                loss = get_loss(self.t_yh_batch, self.a_3)
                
                train_acc_log_tmp.append(acc)
                train_loss_log_tmp.append(loss)
                
                self.update_weights(self.t_yh_batch,self.a_3)
            
            


            train_acc = np.array(train_acc_log_tmp).mean()
            #train_loss = np.array(train_loss_log_tmp).mean()
            train_loss = 0
            print("epoch : %s, loss : %s, tra_acc : %s"%(epoch,train_loss,train_acc))
            
           

            # load validation images of the batch size for every iteration
            for i, data in enumerate(valloader):

                # inputs is the image
                # labels is the class of the image
                inputs, labels = data
                inputs, labels = inputs.float() , labels.float()
                # if you don't change the image size, it will be [batch_size, 1, 100, 100]
                
                
                # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
                
                self.v_data_batch = torch.squeeze(torch.FloatTensor(data[0]),3)
                
                self.v_data_batch = self.v_data_batch.view(len(data[0]),FEATURE_SIZE).to(device)
                
                self.z_1 = torch.matmul(self.v_data_batch,self.w_1.T) + self.b_1
                
                self.a_1 = get_activation(self.z_1,self.act_type_1)
                #print(self.a_1)

                self.z_2 = torch.matmul(self.a_1,self.w_2.T) + self.b_2
                self.a_2 = get_activation(self.z_2,self.act_type_2) # 1027 x 50

                self.z_3 = torch.matmul(self.a_2,self.w_3.T) + self.b_3
                self.a_3 = get_activation(self.z_3,self.act_type_3) # 1027 x 1

                self.v_yh_batch = data[1].float().unsqueeze(1).to(device)
                acc = self.get_acc(self.a_3,self.v_yh_batch)
                
                
                loss = get_loss(self.v_yh_batch, self.a_3)
                val_acc_log_tmp.append(acc)
                val_loss_log_tmp.append(loss)
            val_acc = np.array(val_acc_log_tmp).mean()
            #val_loss = np.array(val_loss_log_tmp).mean()
            val_loss = 0
            print("epoch : %s, loss : %s, val_acc : %s"%(epoch,val_loss,val_acc))

            self.train_acc_log.append(train_acc)
            self.train_loss_log.append(train_loss)
            self.val_acc_log.append(val_acc)
            self.val_loss_log.append(val_loss)
            self.epoch_log.append(epoch)
            epoch += 1
            
            tmp_idx = len(self.train_loss_log)-1
            if ( abs(self.train_loss_log[tmp_idx]-self.train_loss_log[tmp_idx-1]) < self.min_loss_diff) :
                print("Learning is terminated.")
                break

        
    def update_weights(self,t_y,a_3) :
        error_wb3 = -(torch.div(t_y,a_3+ 0.00011) - torch.div(1.0-t_y,1.0-a_3+ 0.00001)) # sum ep
        d_z_3 = error_wb3*get_derv_activation(self.z_3,self.act_type_3) #         
        d_w_3 = torch.matmul(d_z_3.T,self.a_2)
        d_b_3 = torch.sum(d_z_3, dim=0, keepdim=True) / self.a_2.shape[0] # mean도 됨
        
        #########
        
        error_wb2 = torch.matmul(d_z_3,self.w_3)
        
        d_z_2 = error_wb2*get_derv_activation(self.z_2,self.act_type_2)
        
        d_w_2 = torch.matmul(d_z_2.T,self.a_1)
        d_b_2 = torch.sum(d_z_2, dim=0,keepdims=True) / self.a_1.shape[0]
        
   
        error_wb1 = torch.matmul(d_z_2,self.w_2)
        d_z_1 = error_wb1*get_derv_activation(self.z_1,self.act_type_1)
        d_w_1 = torch.matmul(d_z_1.T,self.t_data_batch)
        d_b_1 = torch.sum(d_z_1, dim=0,keepdims=True) / self.t_data_batch.shape[0]
        
        
        self.w_3 += -self.lr*d_w_3
        self.b_3 += -self.lr*d_b_3
        
        self.w_2 += -self.lr*d_w_2
        self.b_2 += -self.lr*d_b_2
        self.w_1 += -self.lr*d_w_1
        self.b_1 += -self.lr*d_b_1
        #print(b_3,b_2,b_1)
        
    def get_acc(self,yhat,y) :
        count = 0
      
        for a,b in zip(yhat,y) :
            if a >= 0.5 :
                if b == 1 :
                    count+=1
            else :
                if b == 0:
                    count +=1
        
        return count / len(yhat)

    def show_loss(self) :
        #print(self.train_loss_log)
        #print(self.val_loss_log)
        #print(self.epoch_log)
        
        tmp_1 = torch.tensor(self.train_loss_log)
        tmp_2 = torch.tensor(self.epoch_log)
       
        t1 = plt.plot(self.epoch_log,self.train_loss_log, color='orange',label='Training Loss')
        t2 = plt.plot(self.epoch_log,self.val_loss_log, color= 'red',label='Validation Loss')
        plt.title("Loss")
        plt.legend(['Training Loss','Validation Loss'])
        plt.show()
    def show_acc(self) :
        t1 = plt.plot(self.epoch_log,self.train_acc_log, color='orange',label='Training Acc')
        t2 = plt.plot(self.epoch_log,self.val_acc_log, color= 'red',label='Validation Acc')
        plt.title("Accuracy")
        plt.legend(['Training Acc','Validation Acc'])
        plt.show()



In [ ]:

activation_type = 0 # 0 = sigmoid
learningRate = 0.001
min_loss_diff = 0
machine = ML(0,0,0,10,5,1,learningRate,min_loss_diff)
dtype = torch.FloatTensor


ML Object initialized


In [ ]:
machine.training()

epoch : 0, loss : 0, tra_acc : 0.488956228956229
epoch : 0, loss : 0, val_acc : 0.4973809523809524
epoch : 1, loss : 0, tra_acc : 0.4748484848484848
epoch : 1, loss : 0, val_acc : 0.4992857142857143
epoch : 2, loss : 0, tra_acc : 0.5040404040404041
epoch : 2, loss : 0, val_acc : 0.4311904761904762
epoch : 3, loss : 0, tra_acc : 0.6202356902356903
epoch : 3, loss : 0, val_acc : 0.4395238095238095
epoch : 4, loss : 0, tra_acc : 0.6351515151515152
epoch : 4, loss : 0, val_acc : 0.43357142857142855
epoch : 5, loss : 0, tra_acc : 0.6356902356902357
epoch : 5, loss : 0, val_acc : 0.425
epoch : 6, loss : 0, tra_acc : 0.6482491582491582
epoch : 6, loss : 0, val_acc : 0.4342857142857142
epoch : 7, loss : 0, tra_acc : 0.6485185185185185
epoch : 7, loss : 0, val_acc : 0.4440476190476191
epoch : 8, loss : 0, tra_acc : 0.6424242424242425
epoch : 8, loss : 0, val_acc : 0.4578571428571429
epoch : 9, loss : 0, tra_acc : 0.6561616161616161
epoch : 9, loss : 0, val_acc : 0.44285714285714284
epoch : 10, 

epoch : 82, loss : 0, tra_acc : 0.7643434343434343
epoch : 82, loss : 0, val_acc : 0.6126190476190477
epoch : 83, loss : 0, tra_acc : 0.7700673400673401
epoch : 83, loss : 0, val_acc : 0.6602380952380952
epoch : 84, loss : 0, tra_acc : 0.7861616161616162
epoch : 84, loss : 0, val_acc : 0.6464285714285715
epoch : 85, loss : 0, tra_acc : 0.7791582491582492
epoch : 85, loss : 0, val_acc : 0.6402380952380953
epoch : 86, loss : 0, tra_acc : 0.794074074074074
epoch : 86, loss : 0, val_acc : 0.6699999999999999
epoch : 87, loss : 0, tra_acc : 0.7944444444444443
epoch : 87, loss : 0, val_acc : 0.665
epoch : 88, loss : 0, tra_acc : 0.7822558922558923
epoch : 88, loss : 0, val_acc : 0.6342857142857143
epoch : 89, loss : 0, tra_acc : 0.7848821548821548
epoch : 89, loss : 0, val_acc : 0.6609523809523808
epoch : 90, loss : 0, tra_acc : 0.7962626262626262
epoch : 90, loss : 0, val_acc : 0.6569047619047619
epoch : 91, loss : 0, tra_acc : 0.7965319865319865
epoch : 91, loss : 0, val_acc : 0.60714285714

epoch : 162, loss : 0, tra_acc : 0.8595286195286196
epoch : 162, loss : 0, val_acc : 0.7428571428571429
epoch : 163, loss : 0, tra_acc : 0.8811784511784513
epoch : 163, loss : 0, val_acc : 0.7730952380952382
epoch : 164, loss : 0, tra_acc : 0.8598989898989898
epoch : 164, loss : 0, val_acc : 0.8116666666666666
epoch : 165, loss : 0, tra_acc : 0.8571717171717171
epoch : 165, loss : 0, val_acc : 0.6995238095238095
epoch : 166, loss : 0, tra_acc : 0.8643434343434344
epoch : 166, loss : 0, val_acc : 0.7035714285714286
epoch : 167, loss : 0, tra_acc : 0.8566329966329967
epoch : 167, loss : 0, val_acc : 0.6980952380952381
epoch : 168, loss : 0, tra_acc : 0.8574410774410773
epoch : 168, loss : 0, val_acc : 0.7102380952380952
epoch : 169, loss : 0, tra_acc : 0.8708080808080808
epoch : 169, loss : 0, val_acc : 0.7733333333333333
epoch : 170, loss : 0, tra_acc : 0.8714478114478115
epoch : 170, loss : 0, val_acc : 0.7647619047619049
epoch : 171, loss : 0, tra_acc : 0.8635353535353535
epoch : 171,

epoch : 242, loss : 0, tra_acc : 0.8938047138047138
epoch : 242, loss : 0, val_acc : 0.728095238095238
epoch : 243, loss : 0, tra_acc : 0.9008080808080808
epoch : 243, loss : 0, val_acc : 0.7214285714285714
epoch : 244, loss : 0, tra_acc : 0.9023569023569024
epoch : 244, loss : 0, val_acc : 0.7088095238095238
epoch : 245, loss : 0, tra_acc : 0.90989898989899
epoch : 245, loss : 0, val_acc : 0.7207142857142856
epoch : 246, loss : 0, tra_acc : 0.9154545454545455
epoch : 246, loss : 0, val_acc : 0.7133333333333333
epoch : 247, loss : 0, tra_acc : 0.9109090909090909
epoch : 247, loss : 0, val_acc : 0.7028571428571428
epoch : 248, loss : 0, tra_acc : 0.8998989898989899
epoch : 248, loss : 0, val_acc : 0.7169047619047619
epoch : 249, loss : 0, tra_acc : 0.9005387205387205
epoch : 249, loss : 0, val_acc : 0.7088095238095239
epoch : 250, loss : 0, tra_acc : 0.9074410774410773
epoch : 250, loss : 0, val_acc : 0.7233333333333333
epoch : 251, loss : 0, tra_acc : 0.9047138047138046
epoch : 251, lo

epoch : 322, loss : 0, tra_acc : 0.9441750841750841
epoch : 322, loss : 0, val_acc : 0.708095238095238
epoch : 323, loss : 0, tra_acc : 0.9417171717171716
epoch : 323, loss : 0, val_acc : 0.708095238095238
epoch : 324, loss : 0, tra_acc : 0.942996632996633
epoch : 324, loss : 0, val_acc : 0.7095238095238097
epoch : 325, loss : 0, tra_acc : 0.9417171717171716
epoch : 325, loss : 0, val_acc : 0.7245238095238097
epoch : 326, loss : 0, tra_acc : 0.9490909090909092
epoch : 326, loss : 0, val_acc : 0.7133333333333333
epoch : 327, loss : 0, tra_acc : 0.95
epoch : 327, loss : 0, val_acc : 0.6921428571428571
epoch : 328, loss : 0, tra_acc : 0.9426262626262626
epoch : 328, loss : 0, val_acc : 0.7095238095238096
epoch : 329, loss : 0, tra_acc : 0.946902356902357
epoch : 329, loss : 0, val_acc : 0.7252380952380952
epoch : 330, loss : 0, tra_acc : 0.9484511784511785
epoch : 330, loss : 0, val_acc : 0.7133333333333334
epoch : 331, loss : 0, tra_acc : 0.9441750841750842
epoch : 331, loss : 0, val_acc

...

epoch : 2137, loss : 0, tra_acc : 0.9927272727272727
epoch : 2137, loss : 0, val_acc : 0.7366666666666667
epoch : 2138, loss : 0, tra_acc : 0.9902693602693605
epoch : 2138, loss : 0, val_acc : 0.7209523809523809
epoch : 2139, loss : 0, tra_acc : 0.9927272727272727
epoch : 2139, loss : 0, val_acc : 0.7419047619047618
epoch : 2140, loss : 0, tra_acc : 0.9927272727272727
epoch : 2140, loss : 0, val_acc : 0.7366666666666667
epoch : 2141, loss : 0, tra_acc : 0.9927272727272727
epoch : 2141, loss : 0, val_acc : 0.7366666666666667
epoch : 2142, loss : 0, tra_acc : 0.9927272727272727
epoch : 2142, loss : 0, val_acc : 0.7445238095238095
epoch : 2143, loss : 0, tra_acc : 0.9927272727272727
epoch : 2143, loss : 0, val_acc : 0.734047619047619
epoch : 2144, loss : 0, tra_acc : 0.9927272727272727
epoch : 2144, loss : 0, val_acc : 0.7419047619047618
epoch : 2145, loss : 0, tra_acc : 0.9927272727272727
epoch : 2145, loss : 0, val_acc : 0.7419047619047618
epoch : 2146, loss : 0, tra_acc : 0.99026936026

epoch : 2415, loss : 0, tra_acc : 0.9927272727272727
epoch : 2415, loss : 0, val_acc : 0.734047619047619
epoch : 2416, loss : 0, tra_acc : 0.9927272727272727
epoch : 2416, loss : 0, val_acc : 0.734047619047619
epoch : 2417, loss : 0, tra_acc : 0.9927272727272727
epoch : 2417, loss : 0, val_acc : 0.747142857142857
epoch : 2418, loss : 0, tra_acc : 0.9927272727272727
epoch : 2418, loss : 0, val_acc : 0.7392857142857143
epoch : 2419, loss : 0, tra_acc : 0.9902693602693603
epoch : 2419, loss : 0, val_acc : 0.7392857142857143
epoch : 2420, loss : 0, tra_acc : 0.9927272727272727
epoch : 2420, loss : 0, val_acc : 0.7314285714285714
epoch : 2421, loss : 0, tra_acc : 0.9927272727272727
epoch : 2421, loss : 0, val_acc : 0.734047619047619
epoch : 2422, loss : 0, tra_acc : 0.9927272727272727
epoch : 2422, loss : 0, val_acc : 0.7445238095238095
epoch : 2423, loss : 0, tra_acc : 0.9927272727272727
epoch : 2423, loss : 0, val_acc : 0.734047619047619
epoch : 2424, loss : 0, tra_acc : 0.992727272727272

epoch : 2493, loss : 0, tra_acc : 0.9927272727272727
epoch : 2493, loss : 0, val_acc : 0.7399999999999999
epoch : 2494, loss : 0, tra_acc : 0.9927272727272727
epoch : 2494, loss : 0, val_acc : 0.7478571428571428
epoch : 2495, loss : 0, tra_acc : 0.9927272727272727
epoch : 2495, loss : 0, val_acc : 0.7347619047619047
epoch : 2496, loss : 0, tra_acc : 0.9927272727272727
epoch : 2496, loss : 0, val_acc : 0.7399999999999999
epoch : 2497, loss : 0, tra_acc : 0.9902693602693603
epoch : 2497, loss : 0, val_acc : 0.7321428571428571
epoch : 2498, loss : 0, tra_acc : 0.9927272727272727
epoch : 2498, loss : 0, val_acc : 0.7399999999999999
epoch : 2499, loss : 0, tra_acc : 0.9927272727272727
epoch : 2499, loss : 0, val_acc : 0.7347619047619047
epoch : 2500, loss : 0, tra_acc : 0.9927272727272727
epoch : 2500, loss : 0, val_acc : 0.7373809523809524
epoch : 2501, loss : 0, tra_acc : 0.9927272727272727
epoch : 2501, loss : 0, val_acc : 0.7314285714285714
epoch : 2502, loss : 0, tra_acc : 0.9927272727

epoch : 2571, loss : 0, tra_acc : 0.9927272727272727
epoch : 2571, loss : 0, val_acc : 0.7452380952380953
epoch : 2572, loss : 0, tra_acc : 0.9927272727272727
epoch : 2572, loss : 0, val_acc : 0.7269047619047618
epoch : 2573, loss : 0, tra_acc : 0.9927272727272727
epoch : 2573, loss : 0, val_acc : 0.7295238095238096
epoch : 2574, loss : 0, tra_acc : 0.9927272727272727
epoch : 2574, loss : 0, val_acc : 0.7242857142857143
epoch : 2575, loss : 0, tra_acc : 0.9902693602693603
epoch : 2575, loss : 0, val_acc : 0.7452380952380953
epoch : 2576, loss : 0, tra_acc : 0.9927272727272727
epoch : 2576, loss : 0, val_acc : 0.7399999999999999
epoch : 2577, loss : 0, tra_acc : 0.9927272727272727
epoch : 2577, loss : 0, val_acc : 0.7399999999999999
epoch : 2578, loss : 0, tra_acc : 0.9927272727272727
epoch : 2578, loss : 0, val_acc : 0.7373809523809524
epoch : 2579, loss : 0, tra_acc : 0.9927272727272727
epoch : 2579, loss : 0, val_acc : 0.7504761904761904
epoch : 2580, loss : 0, tra_acc : 0.9927272727

epoch : 2649, loss : 0, tra_acc : 0.9927272727272727
epoch : 2649, loss : 0, val_acc : 0.7426190476190476
epoch : 2650, loss : 0, tra_acc : 0.9902693602693603
epoch : 2650, loss : 0, val_acc : 0.7452380952380953
epoch : 2651, loss : 0, tra_acc : 0.9927272727272727
epoch : 2651, loss : 0, val_acc : 0.7295238095238096
epoch : 2652, loss : 0, tra_acc : 0.9927272727272727
epoch : 2652, loss : 0, val_acc : 0.7295238095238096
epoch : 2653, loss : 0, tra_acc : 0.9927272727272727
epoch : 2653, loss : 0, val_acc : 0.7347619047619047
epoch : 2654, loss : 0, tra_acc : 0.9927272727272727
epoch : 2654, loss : 0, val_acc : 0.7261904761904763
epoch : 2655, loss : 0, tra_acc : 0.9927272727272727
epoch : 2655, loss : 0, val_acc : 0.7321428571428571
epoch : 2656, loss : 0, tra_acc : 0.9927272727272727
epoch : 2656, loss : 0, val_acc : 0.7530952380952382
epoch : 2657, loss : 0, tra_acc : 0.9927272727272727
epoch : 2657, loss : 0, val_acc : 0.7295238095238096
epoch : 2658, loss : 0, tra_acc : 0.9902693602

In [ ]:
machine.show_loss()

In [ ]:
machine.show_acc()